In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_aws import ChatBedrock
import re
import json

In [2]:
print("langchain advanced")

langchain advanced


In [3]:
region = boto3.Session().region_name
print(region)

us-west-2


In [4]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=region 
)

In [5]:
#llm
llm = ChatBedrock(
    client=bedrock_client,
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
    model_kwargs={
        "max_tokens": 2000,
        "temperature": 0.15,
        "top_p": 0.9,
    }
    )

In [6]:
async def extract_stream(chain, input_json):
    collected_text = []
    async for event in chain.astream_events(input_json):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)


In [7]:
#input { json 포맷 }  -> prompt(text로 변환) -> llm(bedrock) -> output(string타입, json타입)
#여기서 input { json 포맷 }을 활용하는 방법에 대해 이해해보자 ( 여행에 관한 정보를 얻는 llm를 만든다고 가정한다 )

In [8]:
prompt_capital = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 수도에 대해 알려줘")
])

capital_chain = prompt_capital | llm |  StrOutputParser()


# capital_chain.invoke({"country" : "태국"})

In [9]:
async for chunk in capital_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 수도는 **방콕(Bangkok)**입니다.

## 기본 정보
- **정식 명칭**: 크룽텝마하나콘(กรุงเทพมหานคร)
- **인구**: 약 1,000만 명 (수도권 포함)
- **위치**: 태국 중부, 차오프라야 강 유역

## 주요 특징
- **정치·경제 중심지**: 태국의 정치, 경제, 문화의 중심
- **관광 명소**: 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드 등
- **교통**: 수완나품 국제공항, BTS 스카이트레인, MRT 지하철
- **문화**: 전통 태국 문화와 현대 문명이 공존하는 도시

## 유명한 점
- 세계에서 가장 긴 도시 이름을 가진 것으로 유명
- 동남아시아의 주요 허브 도시
- 다양한 음식 문화와 야시장으로 유명
- 불교 사원과 현대적 쇼핑몰이 조화를 이루는 독특한 도시 경관

방콕에 대해 더 자세히 알고 싶은 부분이 있으시면 언제든 물어보세요!

In [10]:
prompt_culture = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 문화에 대해 알려줘")
])

culture_chain = prompt_culture | llm |  StrOutputParser()


# culture_chain.invoke({"country" : "태국"})


In [11]:
async for chunk in culture_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 풍부한 문화에 대해 소개해드리겠습니다.

## 🏛️ **종교와 철학**
- **불교**: 인구의 95%가 소승불교도
- 일상생활에 불교 철학이 깊이 스며들어 있음
- 전국에 4만여 개의 사원(왓)이 있음

## 🙏 **예의와 매너**
- **와이(Wai)**: 합장 인사법으로 상대방에 대한 존경 표현
- 연장자와 지위가 높은 사람에 대한 깊은 존경
- 온화하고 미소 짓는 문화 (미소의 나라)

## 👑 **왕실 문화**
- 왕실에 대한 절대적 존경과 사랑
- 왕실 모독은 법적으로 금지
- 매일 오전 8시, 오후 6시 국가 연주 시 기립

## 🍜 **음식 문화**
- **쌀**: 주식 (카오)
- **향신료**: 매운맛, 신맛, 단맛, 짠맛의 조화
- **대표 음식**: 팟타이, 똠얌꿍, 그린커리, 망고스티키라이스
- 손으로 먹는 문화와 숟가락·포크 사용

## 🎭 **전통 예술**
- **무에타이**: 전통 무술
- **전통 춤**: 라마키엔 등 고전 문학 기반
- **조각과 건축**: 정교한 사원 장식
- **전통 음악**: 가믈란 오케스트라

## 🎉 **축제와 기념일**
- **송크란**: 태국 신정 (4월, 물축제)
- **로이 끄라통**: 등불 축제 (11월)
- **부처님 오신 날**: 위사카 부차

## 👨‍👩‍👧‍👦 **사회 문화**
- 가족 중심의 사회
- 나이와 사회적 지위를 중시하는 위계질서
- '끄렝짜이(배려)'와 '사누크(즐거움)' 중시

태국 문화는 불교 정신을 바탕으로 한 따뜻함과 포용력이 특징입니다.

In [12]:
prompt_climate = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 날씨에 대해 알려줘")
])

climate_chain = prompt_climate | llm |  StrOutputParser()


# climate_chain.invoke({"country" : "태국"})

In [13]:
async for chunk in climate_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 날씨에 대해 설명드리겠습니다.

## 기본 특징
- **열대성 기후**: 연중 고온다습한 날씨
- **평균 기온**: 25-35°C
- **습도**: 연중 높은 습도 (70-80%)

## 계절별 특징

### 🌞 건기 (11월~2월)
- 가장 여행하기 좋은 시기
- 기온: 25-30°C
- 습도가 상대적으로 낮음
- 맑은 날씨가 많음

### 🔥 더위철 (3월~5월)
- 가장 더운 시기
- 기온: 30-40°C
- 매우 덥고 건조함
- 오후에 소나기 가능

### 🌧️ 우기 (6월~10월)
- 몬순 시즌
- 기온: 25-32°C
- 거의 매일 스콜성 비
- 습도가 매우 높음

## 지역별 차이
- **북부** (치앙마이): 상대적으로 서늘, 건기와 우기 구분 뚜렷
- **중부** (방콕): 전형적인 열대기후
- **남부** (푸켓, 코사무이): 연중 고온다습, 지역별로 우기 시기 다름

## 여행 팁
- **최적 시기**: 11월~2월 (건기)
- **준비물**: 가벼운 옷, 우산, 선크림, 모자
- **주의사항**: 우기철 홍수 위험 지역 확인

In [14]:
prompt_location = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 유명한 관광 명소에 대해 알려줘")
])

location_chain = prompt_location | llm |  StrOutputParser()


# climate_location.invoke({"country" : "태국"})

In [15]:
async for chunk in location_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 주요 관광 명소들을 지역별로 소개해드리겠습니다.

## 🏛️ 방콕 (Bangkok)
- **왓 포 (Wat Pho)** - 거대한 와불상으로 유명한 사원
- **왕궁 (Grand Palace)** - 태국 왕실의 화려한 궁전
- **왓 아룬 (Wat Arun)** - 새벽 사원으로 불리는 아름다운 사원
- **차오프라야 강** - 롱테일 보트 투어
- **카오산 로드** - 배낭여행자들의 성지

## 🏖️ 푸켓 (Phuket)
- **파통 비치** - 나이트라이프와 해변 액티비티
- **피피 섬** - 영화 '더 비치' 촬영지
- **빅 부다** - 푸켓의 상징적인 대형 불상
- **올드 타운** - 시노-포르투갈 건축양식의 거리

## 🌴 끄라비 (Krabi)
- **라일레이 비치** - 암벽등반과 에메랄드빛 바다
- **홍 섬** - 카약과 스노클링 명소
- **타이거 케이브 템플** - 1,237개 계단 위의 사원

## 🏔️ 치앙마이 (Chiang Mai)
- **도이 수텝 사원** - 치앙마이 전경을 한눈에
- **올드 시티** - 고대 성벽과 전통 사원들
- **나이트 바자** - 전통 수공예품 쇼핑
- **코끼리 보호구역** - 윤리적 코끼리 체험

## 🏝️ 코사무이 (Koh Samui)
- **빅 부다 템플** - 12미터 높이의 황금 불상
- **나무앙 폭포** - 열대우림 속 폭포
- **차웽 비치** - 가장 인기 있는 해변

각 지역마다 독특한 매력이 있으니 여행 목적에 맞게 선택하시면 좋을 것 같습니다!

In [16]:
#여행에 관한 정보를 얻는 llm를 만든다고 가정한다 위의 네가지 정보를 모두 포함하여 전체적인 형태의 여행 정보를 얻고 싶다고 가정 할때 어떻게 langchain에 활용할수 있을까?
#결국 llm에서 결과가 나오면 string임으로 이 string을 json안에 assgin가능하다

In [17]:
# input에 {"country": "태국"}이라고 두고 itemgetter("country")를 통해 변수를 가져온뒤 각각 llm의 결과를 얻고 sting으로 json에 포함시킨다면? 
# llm을 통과한 결과값들이 새로운 input(json타입)을 형성할수 있지 않을까?
new_input = {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
            }

In [18]:
# new_input의 capital, culture, climate, location가  template_total의 capital, culture, climate, location 변수와 연결된다
template_total = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 가기 가장 좋은 계절을 알려주세요
"""
prompt_total  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_total)

In [19]:
travel_chain = new_input | prompt_total | llm | StrOutputParser()

In [ ]:
async for chunk in travel_chain.astream({"country": "태국"}):
    print(chunk, end="")

In [ ]:
#비슷한 방법으로 RunnableParallel도 존재한다 
parallel_chain = RunnableParallel(
                                  capital=capital_chain, 
                                  culture=culture_chain,
                                  climate=climate_chain, 
                                  location =location_chain
                                 )

In [ ]:
travel_chain_parallel =  parallel_chain | prompt_total | llm | StrOutputParser()

In [ ]:
async for chunk in travel_chain_parallel.astream({"country": "태국"}):
    print(chunk, end="")


In [ ]:
# llm의 outpu의 경우 StrOutputParser()를 활용해서 string으로 나오지만 llm.with_structured_output(obj)을 통해 json포맷으로 받을수 있다 
# llm의 결과가 json포맷으로 오면 이를 활용하여 다른 langchain에 활용할수 있다

In [ ]:
template_plan = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_plan)

In [ ]:
#llm의 결과값 포맷의 schema
class Travel(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    location : str = Field(description="국가 관광명소에 대한 설명")
    plan: str = Field(description="여행계획")


In [ ]:
one_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
)
 

In [ ]:
async for chunk in one_chain.astream({"country": "태국"}):
    print(chunk, end="")


In [ ]:
#data는 llm의 결과 값을 의미합니다  RunnableLambda함수를 활용하면 data와 llm결과를 연결 시켜줄수 있습니다
def transform_keys(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "location": data.location, 
        "plan":     data.plan
    }

In [ ]:
template_detail = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail  = PromptTemplate(input_variables=["capital", "culture", "climate", "location", "plan"],
                                 template=template_detail)

In [ ]:
# RunnableLambda(transform_keys)는 llm과 prompt_detail의 데이터를 매핑 시켜주는 역할
chain_and_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
    | RunnableLambda(transform_keys)
    | prompt_detail
    | llm
    | StrOutputParser()
)


In [ ]:
async for chunk in chain_and_chain.astream({"country": "일본"}):
    print(chunk, end="")


In [ ]:
#목적 다양한 langchain 구조 실험 ( 복잡해도 결국 JSON -> prompt -> LLM -> parser 구조 )

In [ ]:
template_plan_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan_final  = PromptTemplate(input_variables=["capital", "culture", "climate"],
                                 template=template_plan_final)

In [ ]:
class TravelLast(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    plan: str = Field(description="여행계획")

In [ ]:
def transform_key_last(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "plan":     data.plan
    }

In [ ]:
template_detail_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail_final = PromptTemplate(input_variables=["capital", "culture", "climate", "plan"],
                                 template=template_detail_final)

In [ ]:
two_json_chain=({
    "capital": itemgetter("country") | capital_chain,
    "culture": itemgetter("country") | culture_chain,
    "climate": itemgetter("country") | climate_chain
    }
    | prompt_plan_final
    | llm.with_structured_output(TravelLast)
    | RunnableLambda(transform_key_last)
    | {
        "capital": itemgetter("capital"),   # capital("country") -> itemgetter("capital")
        "culture": itemgetter("culture"),   # 쉼표 추가
        "climate": itemgetter("climate"),   # 쉼표 추가
        "plan": itemgetter("plan")
    }
    | prompt_detail_final
    | llm
    | StrOutputParser()
    )


In [ ]:
async for chunk in two_json_chain.astream({"country": "중국"}):
    print(chunk, end="")


In [ ]:
## 목적 결과를 생성하는데 오래 걸리는 langchain일 경우 매 유저가 결과를 기다리는데 어려움을 겪음 
## 따라서 langchain 단계마다 결과를 Stream으로 전달해서 유저가 기다리는 시간을 최소화 시켜주기 위해 만든 코드 
## with_structured_output은 결과가 생성될때 까지 응답을 유저에게 보내지 않기 때문에 사용하지 않음
## 대신 JSON 구조가 포함된 답변을 스트림으로 생성한뒤 LLM 생성이 마치면 모든 스트림을 취합하여 결과를 만들고 그 결과에서 JSON을 발췌하는 형식으로 코드를 구성

In [ ]:
template_plan_streaming = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}

이 정보를 바탕으로 여행 계획을 세워주세요

응답은 반드시 다음 JSON 형식으로 제공해주세요:
{{
    "capital": "수도 정보",
    "culture": "문화 정보", 
    "climate": "기후 정보",
    "plan": "여행 계획"
}}
"""
prompt_plan_streaming  = PromptTemplate(input_variables=["capital", "culture", "climate"],
                                 template=template_plan_streaming)

In [ ]:
info_chain = RunnableParallel({
    "capital": itemgetter("country") | capital_chain,
    "culture": itemgetter("country") | culture_chain,
    "climate": itemgetter("country") | climate_chain
})

# 2단계: 계획 생성
plan_chain = prompt_plan_streaming | llm | StrOutputParser()

# 3단계: 최종 상세 계획
detail_chain = prompt_detail_final | llm | StrOutputParser()


In [ ]:
async def execute_with_progress(country):
    # 1단계 실행 + 출력
    info = await info_chain.ainvoke({"country": country})
    print("✅ 기본 정보 수집 완료:", info)
    
    # 2단계 스트리밍 실행
    print("✅ 여행 계획 생성 중...")
    plan_text = ""
    async for chunk in plan_chain.astream(info):
        print(chunk, end="")
        plan_text += chunk
    
    # JSON 파싱
    json_match = re.search(r'\{.*\}', plan_text, re.DOTALL)
    if json_match:
        plan_data = json.loads(json_match.group())
        plan = type('Plan', (), plan_data)()
    else:
        # 파싱 실패시 기본 구조
        plan = type('Plan', (), {'capital': '', 'culture': '', 'climate': '', 'plan': plan_text.strip()})()
    
    print("\n✅ 여행 계획 생성 완료")
    
    # 3단계 실행
    transformed = transform_key_last(plan)
    async for chunk in detail_chain.astream(transformed):
        print(chunk, end="")


In [ ]:
await execute_with_progress("중국")